In [2]:
########## DEseq2 for Nascent RNA-seq count table ##########
# Author: Huitian (Yolanda) Diao
# April 30th, 2019

######################################## Imports ########################################
library(DESeq2)
library(dplyr)
library(tidyverse)

# Convert names to conditions
cvt_nameToCond <- function(vec_x){
  out_vec <- c()
  for (i in vec_x) {
    out_vec <- c(out_vec, strsplit(i, "_rep")[[1]][1])
  }
  return(out_vec)
}

In [3]:
code.dir <- getwd()
base.dir <- gsub("/codes_local", "", code.dir)

In [4]:
######################################## Config ########################################
wk.dir <- file.path(base.dir, "2_featureCounts_merged")
out.dir <- file.path(base.dir, "3_DEseq2")
out.dir2 <- file.path(base.dir, "3_DEseq2_compiled")
setwd(out.dir)

In [5]:
######################################## Main ########################################
fr.file <- file.path(wk.dir, "Exp337_count_c10.csv")
fr.tb <- read_csv(fr.file) %>% column_to_rownames(var = "gene_name")

###----- Construct DEseq objects
DeseqDesign <- data.frame(names = colnames(fr.tb), condition = as.factor(cvt_nameToCond(colnames(fr.tb))))
DESmat <- DESeqDataSetFromMatrix(countData = fr.tb, colData = DeseqDesign, design = ~ condition)
DESmat <- DESeq(DESmat)


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  gene_name = col_character()
)
ℹ Use `spec()` for the full column specifications.


converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [17]:
norm.df <- data.frame(counts(DESmat, normalized=T))
norm.name <- file.path(out.dir2, "DESeq2_normalized_counts_bysample.csv")
write.csv(norm.df, norm.name)

In [6]:
###----- Write outputs
ko.types <- c("KO_0h", "KO_6h", "KO_24h", "KO_48h")
wt.types <- c("WT_0h", "WT_6h", "WT_24h", "WT_48h")

for (i in ko.types) {
    for (j in ko.types) {
        if (i != j) {
            cts <- c(i,j)
            cts.name <- paste(cts[1], "_vs_", cts[2], ".csv", sep = "")
            cts.res <- results(DESmat, contrast = c("condition", cts))
            write.csv(cts.res, cts.name)
        }
    }
}

for (i in wt.types) {
    for (j in wt.types) {
        if (i != j) {
            cts <- c(i,j)
            cts.name <- paste(cts[1], "_vs_", cts[2], ".csv", sep = "")
            cts.res <- results(DESmat, contrast = c("condition", cts))
            write.csv(cts.res, cts.name)
        }
    }
}

In [7]:
###----- Write outputs
cts.use <- list(c("KO_6h","KO_0h"), c("KO_24h", "KO_0h"), c("KO_48h", "KO_0h"),
                c("WT_6h","WT_0h"), c("WT_24h", "WT_0h"), c("WT_48h", "WT_0h"),
                c("WT_0h", "KO_0h"), c("WT_6h", "KO_6h"), 
                c("WT_24h", "KO_24h"), c("WT_48h", "KO_48h"))
for (i in c(1 : length(cts.use))) {
  cts <- cts.use[[i]]
  cts.name <- paste(cts[1], "_vs_", cts[2], ".csv", sep = "")
  cts.res <- results(DESmat, contrast = c("condition", cts))
  write.csv(cts.res, cts.name)
}


In [6]:
###----- Write outputs
cts.use <- list(c("KO_0h", "WT_0h"), c("KO_6h", "WT_6h"), 
                c("KO_24h", "WT_24h"), c("KO_48h", "WT_48h"))
for (i in c(1 : length(cts.use))) {
  cts <- cts.use[[i]]
  cts.name <- paste(cts[1], "_vs_", cts[2], ".csv", sep = "")
  cts.res <- results(DESmat, contrast = c("condition", cts))
  write.csv(cts.res, cts.name)
}